1. Difference Between AWS Regions, Availability Zones, and Edge Locations
- Term	          ------------Description
 - AWS                              - Region	A physical location in the world with multiple, isolated Availability Zones (e.g., us-east-1, eu-west-1).
 - Availability Zone (AZ)	          - A data center or group of data centers within a region. They offer high availability by being isolated from failures.
 - Edge Location                     -	CDN endpoints used by Amazon


- Importance for Data Analytics & Low-Latency Apps:


    - Deploying close to users (Edge) improves performance.

    - Using multiple AZs ensures high availability.

    - Choosing the right Region ensures compliance (e.g., GDPR in EU).



In [8]:
#2.
!aws ec2 describe-regions --query "Regions[*].RegionName" --output tabl
#2
#AWS CLI: List All Available AWS Regions
!aws ec2 describe-regions --query "Regions[*].RegionName" --output table
#Output (sample):| RegionName   |
#|--------------|
#| us-east-1    |
#| us-west-1    |
#2
#AWS CLI: List All Available AWS Regions
!aws ec2 describe-regions --query "Regions[*].RegionName" --output table
#Output (sample):| RegionName   |
#|--------------|
#| us-east-1    |
#| us-west-1    |
#| eu-central-1 |
#| ap-south-1   |
#| us-east-1    |
#| us-west-1    |
#| eu-central-1 |
#| ap-south-1   |


#| ap-south-1   |


/bin/bash: line 1: aws: command not found
/bin/bash: line 1: aws: command not found
/bin/bash: line 1: aws: command not found


In [9]:
#3
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "s3:ListBucket",
        "s3:GetObject",
        "s3:PutObject"
      ],
      "Resource": [
        "arn:aws:s3:::your-bucket-name",
        "arn:aws:s3:::your-bucket-name/*"
      ]
    }
  ]
}


{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Action': ['s3:ListBucket', 's3:GetObject', 's3:PutObject'],
   'Resource': ['arn:aws:s3:::your-bucket-name',
    'arn:aws:s3:::your-bucket-name/*']}]}

4. Amazon S3 Storage Class Comparison

Storage Class	           Use Case	           Retrieval time        COST
Standard           Frequently accessed data   Milliseconds  $$(highest
Intelligent-Tiering	Unknown access patterns; auto-tiering	Milliseconds	$$
Glacier	Archival data; infrequent access	Minutes to hrs	$

✅ Best Practice:

Use Standard for active analytics.

Use Intelligent-Tiering for unpredictable usage.

Use Glacier for archived logs or compliance storage.

5. RDS vs DynamoDB vs Redshift in Data Pipelines
Service	Type	Best For	Example Use Case
Amazon RDS	Relational (SQL)	Structured transactional data	Store user info or metadata for analytics apps
Amazon DynamoDB	NoSQL (Key-Value)	Real-time, high-speed lookups	Track user behavior or logs in dashboards
Amazon Redshift	Data Warehouse	Large-scale analytical queries	OLAP queries on sales, customer, product data

6. What is Serverless Computing? Pros & Cons of AWS Lambda
🔍 Definition:
Serverless computing abstracts infrastructure management. You deploy code and AWS automatically handles provisioning, scaling, and maintenance.

✅ Pros:
No server management

Auto scaling

Pay only for usage

Easy to connect with AWS services

❌ Cons:
Cold start latency

Limited runtime and execution time

Limited debugging

Not ideal for long-running ETL jobs

✅ In Data Pipelines:
AWS Lambda is great for:

Lightweight transformations

Triggers (e.g., on S3 upload)

Real-time notifications

7. Difference Between Kinesis Data Streams, Firehose, and Analytics
Service	Description	Use Case
Kinesis Data Streams	Real-time stream capture (DIY processing)	Custom app to process sensor or log data
Kinesis Firehose	Managed delivery stream → S3/Redshift	Real-time ETL to S3 or Redshift
Kinesis Data Analytics	SQL-based stream analytics	Analyze clickstream in real-time

8. Columnar Storage and Redshift Performance
🔍 Columnar Storage:
Stores data column-by-column rather than row-by-row.

✅ Benefits in Redshift:
Reduces I/O for analytic queries (only reads relevant columns)

Enables better compression

Improves query performance for large data scans

✅ Why Important:
For OLAP-style queries like SELECT SUM(sales) FROM orders, columnar access dramatically reduces read time and improves speed

1. S3 Bucket Creation and Versioning

Created bucket: analytics-demo-bucket

Enabled versioning via S3 console.

Uploaded sales.csv twice with changes to show 2 versions.

Screenshot Placeholder: s3_versioning_screenshot.png


In [10]:
#2. Lifecycle Policy for Glacier + Deletion

#Policy JSON:
{
  "Rules": [
    {
      "ID": "MoveToGlacierAndDelete",
      "Status": "Enabled",
      "Filter": {"Prefix": ""},
      "Transitions": [
        {
          "Days": 30,
          "StorageClass": "GLACIER"
        }
      ],
      "Expiration": {
        "Days": 90
      }
    }
  ]
}

{'Rules': [{'ID': 'MoveToGlacierAndDelete',
   'Status': 'Enabled',
   'Filter': {'Prefix': ''},
   'Transitions': [{'Days': 30, 'StorageClass': 'GLACIER'}],
   'Expiration': {'Days': 90}}]}

In [14]:
#3. DynamoDB Table and Lambda Trigger

#Table: CustomerData

#Primary key: customer_id

#Inserted 3 manual records.

#Lambda Code: (S3-triggered
# Install boto3 if it's not already installed
!pip install boto3
import json
import boto3

def lambda_handler(event, context):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('CustomerData')
    for record in event['Records']:
        table.put_item(Item={
            'customer_id': record['s3']['object']['key'],
            'timestamp': record['eventTime']
        })
        return 'Successfully processed {} records.'.format(len(event['Records']))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 6.0 MB/s eta 0:00:00


In [19]:

#4. Lambda Logging to CloudWatch

#Trigger: S3 upload

#Logs filename, size, and timestamp

#Code Snippet:
import json
import logging

def lambda_handler(event, context):
    for record in event['Records']:
        print("File Uploaded:", record['s3']['object']['key'])
        print("Size:", record['s3']['object']['size'])
        print("Timestamp:", record['eventTime'])

        #Screenshot Placeholder: cloudwatch_logs_upload.png

In [35]:
#5 Crawled S3 bucket → Created table in Glue Data Catalog

# Ran job to convert sales.csv to sales.parquet

#Glue Job Script:

dyf = glueContext.create_dynamic_frame.from_catalog(database="demo_db", table_name="sales_csv")
dyf_parquet = dyf.repartition(1)
    frame = dyf_parquet,
    connection_type = "s3",
    connection_options = {"path": "s3://analytics-demo-bucket/output/"},
    format = "parquet"

)
CREATE TABLE sales (
    id INT,
    date DATE,
    amount FLOAT
);
COPY sales FROM 's3://analytics-demo-bucket/sales.csv'
CREDENTIALS 'aws_iam_role=arn:aws:iam::account-id:role/RedshiftRole'
CSV;
SELECT * FROM "demo_db"."sales_csv" LIMIT 10;

IndentationError: unexpected indent (<ipython-input-35-816d58cb5fca>, line 9)

End-to-End Data Analytics Pipeline on AWS
This pipeline demonstrates a common pattern for batch processing and analytics on AWS.

mermaid
graph LR
    A[Data Source] --> B(Ingestion: Kinesis Firehose);
    B --> C(Storage: S3 Data Lake);
    C --> D(Transformation: Glue ETL);
    D --> E(Analytics Data Store: Redshift);
    D --> F(NoSQL Data Store: DynamoDB);
    E --> G(Visualization/BI: QuickSight);
    F --> G;
    C --> H(Ad-hoc Querying: Athena);
    H --> G;
Use code with caution
Pipeline Stages and Service Choices
Data Ingestion: Kinesis Firehose

Why Kinesis Firehose? Kinesis Firehose is a fully managed service for reliably loading streaming data into data lakes, data stores, and analytics services. It's ideal for this stage because it simplifies the process of capturing, transforming, and loading streaming data without requiring you to manage infrastructure. It automatically scales to handle varying data volumes and can batch and compress data before sending it to the destination, optimizing for cost and downstream processing.
How it's used: Data streams from various sources (applications, IoT devices, logs) are sent to Kinesis Firehose. Firehose automatically buffers the data and delivers it to the designated S3 bucket.
Data Storage: S3 Data Lake

Why S3? Amazon S3 (Simple Storage Service) is an object storage service offering industry-leading scalability, data availability, security, and performance. It's the foundational layer of a data lake due to its low cost, durability, and ability to store structured, semi-structured, and unstructured data in its raw format. This allows for schema-on-read flexibility later in the pipeline.
How it's used: Kinesis Firehose delivers the raw incoming data directly into specified folders within an S3 bucket. This raw data layer serves as the single source of truth for the analytics pipeline.
Data Transformation: Glue ETL

Why Glue ETL? AWS Glue is a fully managed extract, transform, and load (ETL) service that makes it easy to prepare and load your data for analytics. It's serverless, meaning you don't need to manage servers. Glue Data Catalog can automatically discover and catalog metadata from the S3 data lake, providing a unified metadata repository. Glue ETL jobs (written in Python or Scala) can read data from S3, perform transformations (cleaning, joining, aggregating, changing format, partitioning), and write the transformed data to another location, such as a curated layer in S3 or directly into Redshift.
How it's used: A Glue Crawler can scan the raw data in S3 and populate the Glue Data Catalog with table definitions. A Glue ETL job reads data from the raw S3 layer (using the Data Catalog), applies necessary transformations (e.g., converting CSV to Parquet or ORC for better query performance, filtering data, enriching records), and writes the clean, transformed data to a curated S3 layer or directly loads it into Redshift.
Analytics Data Store: Redshift

Why Redshift? Amazon Redshift is a fully managed, petabyte-scale data warehouse service. It's optimized for large-scale analytical queries (OLAP - Online Analytical Processing) on structured and semi-structured data. Redshift uses columnar storage and parallel processing, making it significantly faster than traditional row-based databases for analytical workloads. It's ideal for powering dashboards and business intelligence tools.
How it's used: The transformed data from the Glue ETL job can be loaded into Redshift for fast querying and reporting by BI tools. Redshift Spectrum can also be used to query data directly in S3 without loading it, providing flexibility.
NoSQL Data Store (for specific use cases): DynamoDB

Why DynamoDB? Amazon DynamoDB is a fast and flexible NoSQL database service for single-digit millisecond performance at any scale. While not the primary store for broad analytics, it's excellent for specific use cases requiring low-latency key-value lookups, such as storing session data, user profiles, feature flags, or real-time dashboards powered by pre-aggregated data.
How it's used: Data processed by Lambda or Glue might update or populate DynamoDB for real-time applications or specific dashboard widgets that require fast access to individual items.
Ad-hoc Querying: Athena

Why Athena? Amazon Athena is an interactive query service that makes it easy to analyze data directly in Amazon S3 using standard SQL. It's serverless, and you pay only for the queries you run. Athena works directly with the Glue Data Catalog and is perfect for data analysts or data scientists who need to explore data in S3 without needing to load it into a data warehouse.
How it's used: Analysts can use Athena to query the raw or transformed data stored in S3 using SQL, making it easy to perform ad-hoc analysis or explore data quality before loading into Redshift.
Visualization/Business Intelligence: QuickSight

Why QuickSight? Amazon QuickSight is a scalable, serverless, embeddable, machine learning-powered business intelligence (BI) service built for the cloud. It allows you to easily create and publish interactive dashboards, reports, and analytics. QuickSight can connect directly to various AWS data sources, including Redshift, Athena, S3, and DynamoDB (via other services if needed), making it a natural choice for visualizing the results of your data pipeline.
How it's used: QuickSight connects to Redshift (for curated analytical data), Athena (for querying data in S3), or potentially DynamoDB (for specific real-time data points) to build dashboards and visualizations that provide business insights.
This pipeline provides a robust, scalable, and cost-effective way to handle batch data analytics workloads on AWS, leveraging managed services to reduce operational overhead.